**<font size=5 color="aa7700">Welcome to Food and Restaurants EDA</font>**<br/><br/><br/>
<span style="color:#D4AF37">
***<font size=3><u>Sections of Restaurants' EDA</u></font>***
<br/>

<A href="#section1"><font color='blue' size=4>Global</font></A>
<A href="#section1.1">1. What are the some Cities around the globe with Maximum Restaurants?</A><br/>
<A href="#section1.2">2. What are the Top10 Popular Ratings as per the Counts?</A><br/>
<A href="#section1.3">3. Word Cloud Visualization: for the Cuisines (as per its frequencies)</A><br/>
<A href="#section1.4">4. What are the Top20 Cuisines?</A><br/>
<A href="#section1.5">5. Where are the Restaurants hiding & what's their Ratings?</A><br/>
<A href="#section1.6">6. Most Common Ingredients used in Cuisines Worldwide!</A><br/>
<A href="#section1.7">7. Top Comfort Foods</A><br/>
<A href="#section1.8">8. Popular Comfort Food choices when you are Stressed out!</A><br/>
<A href="#section1.9">9. Is there a relation between Average GPA and Food Choices?</A><br/>
<A href="#section1.10">10. How Expensive are the Restaurants?</A><br/>
<A href="#section1.11">11. How does vote effect Aggregate Ratings?</A><br/>
<A href="#section1.12">12.  Eating Out Frequency as per Marital Status</A><br/>
<A href="#section1.13">13. Where are the Starbucks Stores Located ?</A><br/>

<A href="#section2"><font color='blue' size=4>Indian</font></A>
<A href="#section2.1">1. What are the Top10 Indian Cities with Maximum Restaurants?</A><br/>
<A href="#section2.2">2. Word Cloud: for the Restaurant Name (What are some popular Indian Restaurant Names)?</A><br/>
<A href="#section2.3">3. How are the Rating types ('Excellent', 'Poor' etc) represented as per the Average Ratings (in India)?</A><br/>
<A href="#section2.4">4. Playing with data - asking Questions (just for fun)</A><br/><br/><br/>

<span style="color:#D4AF37">
***<font size=4>Datasets used <font size=4 color='blue'> (multiple sources)</font>***

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt 

#plotly
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()
import plotly.graph_objs as py

#for word cloud
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

import re
import os
print(os.listdir("../input"))
from IPython.display import HTML
from IPython.display import display

from PIL import Image
from termcolor import colored

from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
stop.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','']) # remove it if you need punctuation 

from nltk.stem import WordNetLemmatizer

import seaborn as sns

#to supress Warnings 
import warnings
warnings.filterwarnings("ignore")

**<A name="section1">GLOBAL</A>**


![](https://www.fwdlife.in/wp-content/uploads/2018/05/restaurants.jpg)

In [ ]:
df = pd.read_csv("../input/zomato-restaurants-data/zomato.csv",encoding='latin-1') #Main restaurants data
ri = pd.read_json("../input/recipe-ingredients-dataset/train.json")          #Cuisines recipe ingredients
fc = pd.read_csv("../input/food-choices/food_coded.csv")                     #Food Choice data of college students
sl = pd.read_csv("../input/store-locations/directory.csv")                   #starbucks data
#df.tail(10)

<br/>
**<A name="section1.1"><font color='green'>1. What are the some Cities around the globe with Maximum Restaurants?</font></A>**

In [ ]:
#word cloud - Cities with maximum Restaurants
maskCity = np.array(Image.open( "../input/beerimage/img_482334.png"))

wordcloud = (WordCloud( max_words=100,width=1440, height=1080,max_font_size=60, min_font_size=10, relative_scaling=0.5,mask=maskCity,background_color='white').generate_from_frequencies(df['City'].value_counts()))

fig = plt.figure(1,figsize=(15, 15))
plt.imshow(wordcloud,interpolation="gaussian")
plt.axis('off')
plt.show()

<br/>
**<A name="section1.2"><font color='green'>2. What are the Top10 Popular Ratings as per the Counts?</font></A>**

In [ ]:
TopRatings=df.groupby(['Aggregate rating'],as_index = False)['Restaurant ID'].count()

TopRatings.columns = ['Rating','Counts']
Top = TopRatings.sort_values(by='Counts',ascending=False).head(11).reset_index(drop=True)
Top = Top.iloc[1:,:]
plt.figure(figsize=(15,9))
sns.barplot(Top['Rating'],Top['Counts'])
plt.title("Top 10 Ratings with count")
plt.show()

<br/>
**<A name="section1.3"><font color='green'>3. Word Cloud Visualization: for the Cuisines (as per its frequencies)</font></A>** 

In [ ]:
#Word cloud for cuisines -- lets see some popular cusines 
wave_mask = np.array(Image.open( "../input/beerimage/tulip-beer-glass.jpg"))
wordcloud = (WordCloud(width=1440, height=1080, mask = wave_mask, relative_scaling=0.5, stopwords=stopwords, background_color='grey').generate_from_frequencies(df['Cuisines'].value_counts()))


fig = plt.figure(1,figsize=(15, 15))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

<br/>
**<A name="section1.4"><font color='green'>4. What are the Top20 Cuisines?</font> </A>**

In [ ]:
cuisines_data = df.groupby(['Cuisines'],as_index = False)['Restaurant ID'].count()

#now time to get most popular cusinies on board, we know North Indian gonna hit the list
cuisines_data.columns = ['Popular Cusinies','Number of Restaurants']
cuisines_data.reindex(axis="index")
cuisines_data.sort_values(by='Number of Restaurants',ascending=False).head(20).reset_index(drop=True)


<br/>
**<A name="section1.5">5. Time to see some action on Map:-<font color='green'> Where are the Restaurants hiding & what's their Ratings?</font></A>**

In [ ]:
data1 = [dict(
    type='scattergeo',
    lon = df['Longitude'],
    lat = df['Latitude'],
    text = df['Restaurant Name'],
    mode = 'markers',
    marker = dict(
    cmin = 0,
    color = df['Aggregate rating'],
    cmax = df['Aggregate rating'].max(),
    colorbar=dict(
                title="Ratings"
            )
    )
    
)]
layout = dict(
    title = 'Where are the Resturants',
    hovermode='closest',
    geo = dict(showframe=False, countrywidth=1, showcountries=True,
               showcoastlines=True, showocean = True, projection=dict(type='Mercator'))
)
fig = py.Figure(data=data1, layout=layout)
iplot(fig)

In [ ]:
#Data Prep for below visual
a = []
for i in ri["ingredients"]:
    for j in i:
        a.append(j)
commonIng = pd.DataFrame()
commonIng["common"] = pd.Series(a)
#commonIng

<br/>
**<A name="section1.6"><font color='green'>6. Most 'Common Ingredients' used in Cuisines Worldwide!</font></A>**

In [ ]:
#using recipe-ingredients-dataset
maskCity = np.array(Image.open( "../input/beerimage/carlsberg-beer-650ml-mynepshopcom.jpg"))

wordcloud = (WordCloud(width=1440, height=1080, relative_scaling=0.5, stopwords=stopwords,mask=maskCity,max_words=100,max_font_size=60, min_font_size=10,background_color='black').generate_from_frequencies(commonIng["common"].value_counts()))

fig = plt.figure(1,figsize=(15, 15))
plt.imshow(wordcloud,interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
#Data prep for below visual
a = []
for i in fc["comfort_food"]:
    i = str(i)
    for j in list(i.split(',')):
        a.append(j)
comfy = pd.DataFrame()
comfy["food"] = pd.Series(a)
#comfy["food"]

<br/>
**<A name="section1.7"><font color='green'>7. Top Comfort Foods</font></A>**

In [ ]:
#what's my comfort food -> using Food_choices dataset
maskComfy = np.array(Image.open( "../input/beerimage/images.jpg"))

wordcloud = (WordCloud(width=1440, height=1080, relative_scaling=0.5, stopwords=stopwords,mask=maskComfy,max_words=1000,background_color='white').generate_from_frequencies(comfy["food"].value_counts()))

fig = plt.figure(1,figsize=(15, 15))
plt.imshow(wordcloud,interpolation="gaussian")
plt.axis('off')
plt.show()

In [ ]:
#main functions to get the comfort food sorted from the day as per the reason
"""
1 – stress
2 – boredom
3 – depression/sadness
4 – hunger
5 – laziness
6 – cold weather
7 – happiness 
8- watching tv
"""

def searchComfy(mood):
    lemmatizer = WordNetLemmatizer()
    foodcount = {}
    for i in range(124):
        temp = [temps.strip().replace('.','').replace(',','').lower() for temps in str(fc["comfort_food_reasons"][i]).split(' ') if temps.strip() not in stop ]
        if mood in temp:
            foodtemp = [lemmatizer.lemmatize(temps.strip().replace('.','').replace(',','').lower()) for temps in str(fc["comfort_food"][i]).split(',') if temps.strip() not in stop ]
            for a in foodtemp:
                if a not in foodcount.keys():
                    foodcount[a] = 1 
                else:
                    foodcount[a] += 1
    sorted_food = []
    #print(sorted([(value,key) for (key,value) in foodcount.items()])) : for test
    sorted_food = sorted(foodcount, key=foodcount.get, reverse=True)
    return sorted_food


def findmycomfortfood(mood):
    topn = []
    topn = searchComfy(mood) #function create dictionary only for particular mood
    print(colored("3 Popular Comfort Foods in %s are:"%(mood),'blue'))
    print(colored(topn[0],'green'))
    print(colored(topn[1],'green'))
    print(colored(topn[2],'green'))

<br/>
**<A name="section1.8"><font color='green'>8. Popular Comfort Food choices when you are Stressed out!</font></A>**

In [ ]:
findmycomfortfood('stress')

In [ ]:
#GPA Data Prep
dfc = fc
dfc = dfc.drop(dfc.index[15])
dfc = dfc.drop(dfc.index[60])
dfc = dfc.drop(dfc.index[100])
dfc = dfc.drop(dfc.index[101])
dfc.reset_index(inplace=True)
dfc["GPA"][71]= str(dfc["GPA"][71]).replace(' bitch','')
dfc["GPA"] = pd.to_numeric(dfc["GPA"])

In [ ]:
#main function to plot definition
#coffee: 1-Creamy Frappuccino & 2-Espresso; breakfast: 1-cereal option & 2 – donut option; 
#soup: 1 – veggie soup and 2 – creamy soup; drink: 1 – orange juice and 2 – soda 
from itertools import cycle
cycol = cycle('kbgc')

def barGPA(category,label1, label2):
    barGPA = {} 
    barGPA[label1] = dfc[dfc[category]==1]["GPA"].mean() 
    barGPA[label2] = dfc[dfc[category]==2]["GPA"].mean()
    D = barGPA
    plt.bar(range(len(D)), list(D.values()), align='center',color=next(cycol))
    plt.xticks(range(len(D)), list(D.keys()))
    plt.ylim(ymin=2) 
    plt.yticks(np.arange(2, 3.6, 0.1))
    plt.ylabel("GPA")
    plt.title(category.upper()+" vs GPA")
    plt.grid()

<br/>
**<A name="section1.9"><font color='green'>9. Is there a relation between Average GPA and Food Choices? </font></A>**

In [ ]:
#What can effect your GPA, comparing Average GPA against the choices Students made when asked->
fig = plt.figure(figsize=(12,9))

plt.subplot(2, 2, 1)
barGPA('coffee','Frappuccino','Espresso')

plt.subplot(2, 2, 2)
barGPA('drink','Orange Juice','Soda')


plt.subplot(2, 2, 3)
barGPA('breakfast','Cereal','Donut')

plt.subplot(2, 2, 4)
barGPA('soup','Veggie Soup','Creamy Soup')

plt.show()

<br/>
**<A name="section1.10"><font color='green'>10. How Expensive are the Restaurants? </font></A>**

In [ ]:
#Data prep encoding price range!
dfcopy = df
dfcopy["Price range"] = dfcopy["Price range"].astype(str)
dfcopy["Price range"] = dfcopy["Price range"].str.replace('1', '₹').replace('2', '₹₹').replace('3', '₹₹₹').replace('4', '₹₹₹₹')

In [ ]:
plt.figure(figsize=(15,9))
plt.title("Distribution of Price Ranges")
#plt.bar(x=[1,2,3,4],height=[2,4,8,16],hatch='x',edgecolor=['black']*6)
sns.countplot(dfcopy["Price range"],order=['₹','₹₹','₹₹₹','₹₹₹₹'],hatch='x')
plt.xlabel("Price Range (low to high)",color='green')
plt.ylabel("No. of Restaurants",color='green')
plt.show()

<br/>
**<A name="section1.11"><font color='green'>11. How does vote effect Aggregate Ratings?</font>(seaborn jointplot)</A>**

In [ ]:
#plotting
g = sns.jointplot(x='Aggregate rating', y ='Votes' , data = df,kind='scatter',color='blue')
_ = g.ax_marg_x.hist(df['Aggregate rating'], color="g", alpha=.6)
_ = g.ax_marg_y.hist(df['Votes'], color="y",alpha=.6,
              orientation="horizontal")

<br/>
**<A name="section1.12"><font color='green'>12.  Eating Out Frequency as per Marital Status </font></A>**

In [ ]:
#data prep
"""
1 -Single 
2 - In a relationship 
3 - Cohabiting 
4 - Married 
5 - Divorced 
6 - Widowed
"""
dfc = fc
dfc = dfc.dropna(subset=['marital_status','eating_out'])
dfc['marital_status']=dfc['marital_status'].astype(str)
dfc['marital_status'] = dfc['marital_status'].str.replace('1.0','Single')\
    .replace('2.0','In a relationship').replace('4.0','Married')  #since only 3 kind of Marital status available in data
AvgEatOut = dfc.groupby('marital_status')['eating_out'].mean()

#plotting
plt.figure(figsize=(15,9))
plt.bar(x=AvgEatOut.index, height =AvgEatOut.values , color='white',edgecolor=['red']*3,hatch='*')
plt.xlabel('Marital Status of Students (as per Data)',color='green')
plt.ylabel('Freq. of Eating Out (Average)',color='green')
plt.title("Eating Out Frequency as per Marital Status of Students",color='grey')
plt.show()

<br/>
**<A name="section1.13"><font color='green'>13. Where are the Starbucks Stores Located (around the globe)? </font></A>**

In [ ]:
data1 = [dict(
    type='scattergeo',
    lon = sl['Longitude'],
    lat = sl['Latitude'],
    text = sl['Store Name'],
    mode = 'markers',
    marker = dict(
    color = '#6f4e37',
    )          
    
)]
layout = dict(
    title = 'Where is that Starbucks?',titlefont=dict(color='green'), 
    hovermode='closest',
    geo = dict(showframe=False, countrywidth=1, showcountries=True,showocean=True,showland=True,countrycolor='green', 
               showcoastlines=True, projection=dict(type='natural earth')),
    
)
fig = py.Figure(data=data1, layout=layout)
iplot(fig)

In [ ]:
#Checking the Country Code: Indian Country Code is 1
CountryInfo = pd.read_excel('../input/zomato-restaurants-data/Country-Code.xlsx')
#CountryInfo.head()

<br/><br/>
**<A name="section2">INDIAN</A>**


![](http://pluspng.com/img-png/india-png-india-1501.png)

In [ ]:
#taking out Indian Data into new Data frame, which can be reused later
df_india = df[df['Country Code']==1]

TopCities = df_india.groupby(['City'],as_index = False)['Restaurant ID'].count()
TopCities.columns = ['Cities','Number of Restaurants']
Top10 = TopCities.sort_values(by='Number of Restaurants',ascending=False).head(10)



<br/>
**<A name="section2.1"><font color='green'>1. What are the Top 10 Indian Cities with Maximum Restaurants?</font></A>**

In [ ]:
plt.figure(figsize=(20,10))
plt.bar(Top10['Cities'],Top10['Number of Restaurants'], color ='green')
plt.xlabel('Cities')
plt.ylabel('Number of Restaurants')
plt.title('Indian Cities with Maximum Restaurants', fontweight="bold")
plt.show()

<br/>
**<A name="section2.2"><font color='green'>2. Word Cloud: for the Restaurant Name (What are some popular Indian Restaurant Names)?</font></A> **

In [ ]:
#lets see which restaurant got the most shout-out 
#Grey color function
def grey_color_func(word, font_size, position, orientation, random_state=None,
                    **kwargs):
    return "hsl(0, 0%%, %d%%)" % np.random.randint(60, 100)

#Main WordCloud Code
wc = (WordCloud(width=1440, height=1080, relative_scaling=0.5, stopwords=stopwords).generate_from_frequencies(df_india['Restaurant Name'].value_counts()))

fig = plt.figure(1,figsize=(15, 15))
plt.imshow(wc.recolor(color_func=grey_color_func, random_state=3),
           interpolation="bilinear")
plt.axis('off')
plt.show()

<br/>
**<A name="section2.3"><font color='green'>3. How are the Rating types ('Excellent', 'Poor' etc) represented as per the Average Ratings (in India)?</font></A>**

In [ ]:
df["Rating color"].value_counts()
Color_represents = df_india.groupby(['Rating color'],as_index = False)['Aggregate rating'].mean()
Color_represents.columns = ['Rating Color','Average Rating']
Color_represents =Color_represents.sort_values(by='Average Rating',ascending=False)
Color_represents = Color_represents[0:5]
Color_represents['Ratings']  = ['1.Excellent','2.Very Good','3.Good','4.Okay','5.Poor']
#Color_represents

In [ ]:
#Plotting
plt.figure(figsize=(10,5))
a = ['#006400','green','yellow','Orange','Red']
e = ['blue']*5

plt.barh(Color_represents['Ratings'],Color_represents['Average Rating'], align='center', color =a, edgecolor = e, linewidth = 3)
plt.gca().invert_yaxis()
plt.xlabel('Average Rating')
plt.ylabel('Rating Types')
plt.title('Rating Color and the Average Rating they represent', fontweight="bold")
plt.show()

In [ ]:
pd.set_option('display.max_columns', None)   #to display data fully without truncating some columns
df.head(1)
#df['Votes'].mean()
#df['Average Cost for two'].mean()

<br/>
**<A name="section2.4"><font size='3' color='green'>4. Playing with data - asking Questions</font></A> (just for fun, answers are based on the dataset and is subject to change)**<br/>
<font color='green'>Q1.</font> Okay I need some cheap Restaurants, say I have 500 rupees, but i need Restaurant with good Aggregate rating >=4.0 and Average votes (156), I would like to eat some North Indian food; I am in New Delhi, India<br/>
<font color='green'>Q2.</font> Okay Dude, I live in Pune, India. I want to have Chinese Cuisine and I prefer locality to be Viman Nagar, I can spend approax 1000 for two, I wanted Restaurant rated >= 4.0<br/>


In [ ]:
# Q. Okay I need some cheap Restaurants, say I have 500 rupees, but i need Restaurant with good Aggregate rating >=4.0 and Average votes (156), I would like to eat some North Indian food; I am in New Delhi, India
tempDF = df_india[(df_india['Votes']>=156) & (df_india['Aggregate rating'] >= 4.0) & (df_india['Average Cost for two'] <= 500 ) & (df_india['City'] == 'New Delhi') & (df_india['Cuisines']=="North Indian")]
rn = str(tempDF['Restaurant Name'].values[0])
ra = str(tempDF['Address'].values[0])
print(colored("Ans1. ",'green')+colored(rn,'blue')+", Address: "+ra)
#Ans. There you go my pal, Lets hit Jai Vaishno Rasoi

In [ ]:
#Okay Dude, I live in Pune, India. I want to have Chinese Cuisine and I prefer locality to be Viman Nagar, I can spend approax 1000 for two, I wanted Restaurant rated >= 4.0
tempDF = df_india[((df_india['Locality']=='Koregaon Park') & (df_india['Aggregate rating'] >= 4.0) & (df_india['Average Cost for two'] <= 1000 ) & (df_india['City'] == 'Pune'))]
rn = str(tempDF['Restaurant Name'].values[0])
ra = str(tempDF['Address'].values[0])
print(colored("Ans2. ",'green')+colored(rn,'blue')+", Address: "+ra)
#Ans. Is that what you need, if not try-again!

**Thanks for Upvote, your Feedback as comments is highly appreciated :)**